In [1]:
#Application
import cv2
from keras.models import load_model
import numpy as np
from collections import deque

Using TensorFlow backend.
C:\Users\Rittik Grignard\Anaconda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Rittik Grignard\Anaconda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Rittik Grignard\Anaconda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [2]:
model1=load_model(r'C:\Users\Rittik Grignard\devanagari.h5')
print(model1)

In [3]:
letters_count={0:'CHECK',1:'01_ka',2:'02_kha',3:'03_ga',4:'04_gha',5:'05_kna',6:'06_cha',7:'07_chha',8:'08_ja',9:'09_jha',
              10:'10_yna',11:'11_taamataar',12:'12_thaa',13:'13_daa',14:'14_dhha',15:'15_adna',16:'16_tabala',17:'17_tha',
              18:'18_da',19:'19_dha',20:'20_na',21:'21_pa',22:'22_pha',23:'23_ba',24:'24_bha',20:'20_na',25:'25_ma',
              26:'26_yaw',27:'27_ra',28:'28_la',29:'29_waw',30:'30_motosaw',31:'31_petchiryakha',32:'32_patalosaw',
              33:'33_ha',34:'34_chhya',35:'35_tra',36:'36_gya',37:'CHECK'}

In [4]:
def keras_predict(model,image):
    processed=keras_process_image(image)
    print("processed:"+str(processed.shape))
    pred_probab=model.predict(processed)[0]
    ppred_class=list(pred_probab).index(max(pred_probab))
    return max(pred_probab),pred_class

def keras_process_image(img):
    image_x=32
    image_y=32
    img=cv2.resize(img,(image_x,image_y))
    img=np.array(img,dtype=np.float32)
    img=np.reshape(img,(-1,image_x,image_y,1))
    return img

In [ ]:
#streaming
cap=cv2.VideoCapture(0)
Lower_blue=np.array((100,30,30))
Upper_blue=np.array([150,148,255])
pred_class=0
pts=deque(maxlen=512)
blackboard=np.zeros((480,640,3),dtype=np.uint8)
digit=np.zeros((200,200,3),dtype=np.uint8)
while (cap.isOpened()):
    ret,img=cap.read()
    img=cv2.flip(img,1)
    imgHSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(imgHSV,Lower_blue,Upper_blue)
    blur=cv2.medianBlur(mask,15)
    blur=cv2.GaussianBlur(blur,(5,5),0)
    thresh=cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    cnts=cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[1]
    center=None
    if len(cnts)>=1:
        contour=max(cnts,key=cv2.contourArea)
        if cv2.contourArea(contour)>250:
            ((x,y),radius)=cv2.minEnclosingCircle(contour)
            cv2.circle(img,(int(x),int(y)),int(radius),(0,255,255),2)
            cv2.circle(img,center,5,(0,0,255),-1)
            M=cv2.moments(contour)
            center=(int(M['m10']/M['m00']),int(M['m01']/M['m00']))
            pts.appendleft(center)
            for i in range(1,len(pts)):
                if pts[i-1] is None or pts[i] is None:
                    continue
                cv2.line(blackboard,pts[i-1],pts[i],(255,255,255),10)
                cv2.line(img,pts[i-1],pts[i],(0,0,255),5)
    elif len(cnts)==0:
        if len(pts)!=[]:
            blackboard_gray=cv2.cvtColor(blackboard,cv2.COLOR_BGR2GRAY)
            blur1=cv2.medianBlur(blackboard_gray,15)
            blur1=cv2.GaussianBlur(blur1,(5,5),0)
            thresh1=cv2.threshold(blur1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
            blackboard_cnts=cv2.findContours(thresh1.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)[1]
            if len(blackboard_cnts)>=1:
                cnt=max(blackboard_cnts,key=cv2.contourArea)
                print(cv2.contourArea(cnt))
                if cv2.contourArea(cnt)>2000:
                    x,y,w,h=cv2.boundingRect(cnt)
                    digit=blackboard_gray[y:y+h,x:x+w]
                    pred_probab,pred_class=keras_predict(model1,digit)
                    print(pred_class,pred_probab)
        pts=deque(maxlen=512)
        blackboard=np.zeros((480,640,3),dtype=np.uint8)
    cv2.putText(img,"Conv Network:"+str(letters_count[pred_class]),(10,470),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
    cv2.imshow("Frame",img)
    cv2.imshow("Continue",thresh)
    k=cv2.waitKey(10)
    if k==27:
        break
                
    